file nay dung de doc file tdump (toan bo attrs) và ghi file ket qua ra file excel

In [1]:
from units.read import read_data_from_file_like, select_desired_columns
from units.processing import create_datetime_column, create_geodataframe,\
      gdf_lines_and_stats_by_height, split_geodataframe_by_height_code, fill_and_interpolate_gdf
from units.write import write_output, write_output_gdf
import os
import warnings
warnings.filterwarnings("ignore")  # Tắt tất cả các cảnh báo
import pandas as pd
from tqdm import tqdm

In [2]:
folder = 'O:/Tran Xuan Thuy/iWAT/hysplit/output_hysplit_25_3_2025'
files = os.listdir(folder)

In [3]:
len(files)

2316

In [4]:
column_names = [
    'height_code',
    'tmp1',
    'year',
    'month',
    'day',
    'hour',
    'tmp2',
    'tmp3',
    'time_step',
    'n',
    'e',
    'height',
    'pressure',
    'theta',
    'air_temp',
    'rainfall',
    'mixdepth',
    'relhumid',
    'spchumid',
    'h2omixra',
    'terr_msl',
    'sun_flux'
]
desired_cols = [
    'height_code',
    'datetime',
    'time_step',
    'n',
    'e',
    'height',
    'pressure',
    'theta',
    'air_temp',
    'rainfall',
    'mixdepth',
    'relhumid',
    'spchumid',
    'h2omixra',
    'terr_msl',
    'sun_flux'
]

In [5]:
for file in tqdm(files):
    try:
        # --- Example Usage ---
        hour = int(file[-2:])
        day = int(file[-4:-2])
        month = int(file[-6:-4])
        year = 2000 + int(file[-8:-6])
        file_path = os.path.join(folder, file)
        # 1. Reading from a file:
        with open(file_path, "r") as file:
            df = read_data_from_file_like(file, columns_name=column_names)
        # 2. create datetime column
        df = create_datetime_column(df)
        # 3. Selecting desired columns:
        df = select_desired_columns(df, desired_cols)
        # 4. convert to geodataframe -  no geometry needed
        gdf = create_geodataframe(df)
        # 5.1 create lines and stats by height - not calculate mean or sum camulate
        # gdf = gdf_lines_and_stats_by_height(gdf)
        # 5.2 split to 3df by height_code
        gdf1, gdf2, gdf3 = split_geodataframe_by_height_code(gdf)
        # interpolate
        value_cols = ['height', 'pressure', 'theta', 'air_temp', 'rainfall', \
                    'mixdepth', 'relhumid', 'spchumid', 'h2omixra', 'terr_msl', 'sun_flux']
        if gdf1.shape[0] < 241:
            gdf1 = fill_and_interpolate_gdf(gdf1, value_cols)
        if gdf2.shape[0] < 241:
            gdf2 = fill_and_interpolate_gdf(gdf2, value_cols)
        if gdf3.shape[0] < 241:
            gdf3 = fill_and_interpolate_gdf(gdf3, value_cols)
        # # 6 write to file
        file = 'tdump_14060100'
        hh = file[-2:]
        dd = file[-4:-2]
        mm = file[-6:-4]
        yy = 2000 + int(file[-8:-6])
        write_output_gdf(year, month, day, hour, gdf1, gdf2, gdf3,
                         folder_path='O:/Tran Xuan Thuy/iWAT/hysplit/processed_geojson_3103')
    except:
        print('error: {}'.format(file))
        continue

  0%|          | 0/2316 [00:00<?, ?it/s]

100%|██████████| 2316/2316 [05:39<00:00,  6.83it/s]
